# Working with CSV files

In [29]:
# import libraries
import numpy as np
import pandas as pd
from pandas import DataFrame
import itertools
import csv

In [11]:
files = ['DataSets/MallCustomers.csv']

In [33]:
# Read csv files and convert it to data frame
def prepareData(files):
    myData = []
    for file_name in files:
        file = open(file_name, 'rt', encoding='utf-8')
        reviews = csv.reader(file)
        i = 1
        for CustomerID, Genre, Age, AnnualIncome, SpendingScore in reviews:
            if i == 1: i = 2; continue
            myData.append([CustomerID, Genre, Age])
    
    df = DataFrame(myData, columns=['CustomerID', 'Genre', 'Age'])
    return df

In [13]:
myData = prepareData(files)
myData

,CustomerID,Genre,Age
0,1,Male,19
1,2,Male,21
2,3,Female,20
3,4,Female,23
4,5,Female,31
5,6,Female,22
6,7,Female,35
7,8,Female,23
8,9,Male,64
9,10,Female,30


### Load CSV to MySQL database

In [14]:
import mysql.connector
from mysql.connector import errorcode, connect

In [15]:
def connectMysql():
    try:
        con = connect(
            user = "root",
            password = "",
            host = "localhost",
            database = "ds_dataset"
        )
        return con
    except mysql.connector.Error as e:
        if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Access denied !")
        elif e.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exists !")
        else:
            print(e)

In [16]:
def executeSQL(sql):
    try:
        con = connectMysql()
        cursor = con.cursor()
        cursor.execute(sql)
        con.commit()
    except:
        con.rollback()
    con.close()

In [118]:
# Create table
sql_drop = "DROP TABLE IF EXISTS Customers;"
sql_create = "CREATE TABLE Customers(\
CustomerID INT(11) PRIMARY KEY,\
Genre VARCHAR(250),\
Age INT(11));"

executeSQL(sql_drop)
executeSQL(sql_create)

In [129]:
# Insert data from CSV to MySQL
def insertData(CustomerID, Genre, Age):
    sql = "INSERT INTO Customers VALUES(%s, '%s', %s)" % (CustomerID, Genre, Age)
    executeSQL(sql)

for index, (CustomerID, Genre, Age) in myData.iterrows():
    insertData(CustomerID, Genre, Age)

In [22]:
# Get data from database
def getData(sql):
        con = connectMysql()
        cursor = con.cursor()
        cursor.execute(sql)
        reviews = DataFrame(cursor.fetchall())
        reviews.rename(columns = {0 : 'ID', 1 : 'Genre', 2 : 'Age'}, inplace=True)
        cursor.close()
        con.close()
        return reviews

getData("SELECT * FROM Customers;")        

,ID,Genre,Age
0,1,Male,19
1,2,Male,21
2,3,Female,20
3,4,Female,23
4,5,Female,31
5,6,Female,22
6,7,Female,35
7,8,Female,23
8,9,Male,64
9,10,Female,30


### Read CSV using Pandas

In [32]:
files = ['DataSets/MallCustomers.csv']
def modifyCSV():
    for file_path in files:
        df = pd.read_csv(file_path)
        del df['SpendingScore'] # delete column from csv
        del df['Age']
        # export result (csv)
        df.to_csv('DataSets/MallCustomers_afterEditing.csv')
        return df

modifyCSV()

,CustomerID,Genre,AnnualIncome
0,1,Male,15
1,2,Male,15
2,3,Female,16
3,4,Female,16
4,5,Female,17
5,6,Female,17
6,7,Female,18
7,8,Female,18
8,9,Male,19
9,10,Female,19
